### STEP 1: Extraction 

#### 1. Importing Necessary Libraries

In [4]:
import requests
import pandas as pd
import psycopg2
import json

#### 2. Carry out extraction

In [5]:
url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

In [6]:
querystring = {"limit":"500"}

headers = {
	"x-rapidapi-key": "60e9afa955msh7936cf92edeb77ap10b4a1jsnbfc9face381e",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}
response = requests.get(url, headers=headers, params=querystring)

In [6]:
print(response.json())

{'messages': 'The API is unreachable, please contact the API provider', 'info': 'Your Client (working) ---> Gateway (working) ---> API (not working)'}


The API has been shutdown but the Json file was downloaded before the API shutdown, we will read the JSON file from the folder.

In [ ]:
## if the API was working: 
'''
data = response.json()
file_name = 'real_estate.json'

with open(file_name, 'w') as file:
    json.dump(data, file, indent = 4)
'''

In [9]:
real_estate_df = pd.read_json(r'real_estate.json')
real_estate_df.head()

,addressLine1,city,state,zipCode,formattedAddress,bedrooms,squareFootage,yearBuilt,features,county,...,propertyTaxes,lastSalePrice,lastSaleDate,owner,id,longitude,latitude,subdivision,zoning,addressLine2
0,511 John St,Bryan,OH,43506,"511 John St, Bryan, OH 43506",3.0,1404.0,1959.0,"{'architectureType': 'Conventional', 'cooling'...",Williams,...,{'2021': {'total': 1799}},105000.0,2000-10-13T00:00:00.000Z,"{'names': ['SUSAN JO KERR'], 'mailingAddress':...","511-John-St,-Bryan,-OH-43506",-84.570163,41.469595,NaN,NaN,NaN
1,1067 Marlin Dr,Rockledge,FL,32955,"1067 Marlin Dr, Rockledge, FL 32955",3.0,2258.0,1966.0,"{'cooling': True, 'coolingType': 'Central', 'e...",Brevard,...,"{'2021': {'total': 1830}, '2022': {'total': 21...",125000.0,2019-08-13T00:00:00.000Z,"{'names': ['GYULA L KISS', 'JULIE A KISS'], 'm...","1067-Marlin-Dr,-Rockledge,-FL-32955",-80.746982,28.342440,MARLIN SUBD,R2,NaN
2,1136 Sullivan Rd,Ogden,UT,84403,"1136 Sullivan Rd, Apt 15, Ogden, UT 84403",NaN,NaN,NaN,{},Weber,...,NaN,NaN,NaN,NaN,"1136-Sullivan-Rd,-Apt-15,-Ogden,-UT-84403",-111.950330,41.204860,NaN,NaN,Apt 15
3,101 Trinity Lakes Dr,Sun City Center,FL,33573,"101 Trinity Lakes Dr, Apt 361, Sun City Center...",NaN,NaN,NaN,{},Hillsborough,...,NaN,NaN,NaN,NaN,"101-Trinity-Lakes-Dr,-Apt-361,-Sun-City-Center...",-82.355936,27.710284,NaN,NaN,Apt 361
4,7843 S Euclid Ave,Chicago,IL,60649,"7843 S Euclid Ave, Apt 1, Chicago, IL 60649",6.0,3345.0,1925.0,"{'architectureType': 'Two Family', 'cooling': ...",Cook,...,{'2020': {'total': 2527}},225000.0,2022-09-23T00:00:00.000Z,"{'names': ['78TH EUCLID LLC,'], 'mailingAddres...","7843-S-Euclid-Ave,-Apt-1,-Chicago,-IL-60649",-87.576879,41.752132,SOUTHFIELD,NaN,Apt 1


In [10]:
real_estate_df.columns

Index(['addressLine1', 'city', 'state', 'zipCode', 'formattedAddress',
       'bedrooms', 'squareFootage', 'yearBuilt', 'features', 'county',
       'assessorID', 'legalDescription', 'ownerOccupied', 'bathrooms',
       'lotSize', 'propertyType', 'taxAssessment', 'propertyTaxes',
       'lastSalePrice', 'lastSaleDate', 'owner', 'id', 'longitude', 'latitude',
       'subdivision', 'zoning', 'addressLine2'],
      dtype='object')

In [11]:
real_estate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    int64  
 4   formattedAddress  500 non-null    object 
 5   bedrooms          329 non-null    float64
 6   squareFootage     387 non-null    float64
 7   yearBuilt         342 non-null    float64
 8   features          418 non-null    object 
 9   county            500 non-null    object 
 10  assessorID        326 non-null    object 
 11  legalDescription  317 non-null    object 
 12  ownerOccupied     252 non-null    float64
 13  bathrooms         370 non-null    float64
 14  lotSize           321 non-null    float64
 15  propertyType      416 non-null    object 
 16  taxAssessment     318 non-null    object 
 1

### Data Transformation

In [11]:
# Replace missing values with appropriate replacements
real_estate_df.fillna({
    'bedrooms' : 0.0,
    'squareFootage': 0.0,
    'yearBuilt' : 0.0,
    'features' : 'Unknown',
    'assessorID' : 'Unknown',
    'legalDescription' : 'Unknown',
    'ownerOccupied' : 0.0,
    'bathrooms' : 0.0,
    'lotSize' : 0.0,
    'propertyType' : 'Unknown',
    'taxAssessment' : 'Unknown',
    'propertyTaxes' : 'Unknown',
    'lastSalePrice' : 0.0,
    'lastSaleDate' : 'Unknown',
    'zoning' : 'Unknown',
    'addressLine2' : 'Unknown'

}, inplace = True)

In [18]:
real_estate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    int64  
 4   formattedAddress  500 non-null    object 
 5   bedrooms          500 non-null    float64
 6   squareFootage     500 non-null    float64
 7   yearBuilt         500 non-null    float64
 8   features          500 non-null    object 
 9   county            500 non-null    object 
 10  assessorID        500 non-null    object 
 11  legalDescription  500 non-null    object 
 12  ownerOccupied     500 non-null    float64
 13  bathrooms         500 non-null    float64
 14  lotSize           500 non-null    float64
 15  propertyType      500 non-null    object 
 16  taxAssessment     500 non-null    object 
 1

In [14]:

real_estate_df.columns

Index(['addressLine1', 'city', 'state', 'zipCode', 'formattedAddress',
       'bedrooms', 'squareFootage', 'yearBuilt', 'features', 'county',
       'assessorID', 'legalDescription', 'ownerOccupied', 'bathrooms',
       'lotSize', 'propertyType', 'taxAssessment', 'propertyTaxes',
       'lastSalePrice', 'lastSaleDate', 'owner', 'id', 'longitude', 'latitude',
       'subdivision', 'zoning', 'addressLine2'],
      dtype='object')

### DATA MODELLING 

In [32]:
'''
DATA WAS MODELLED IN DRAW.IO FROM THE MODEL WE HAD 3 DIMENSION TABLES AND 1 FACT TABLE
 THE DIMENSION TABLES ARE location_dim, sales_dim, features_dim
 THE FACT TABLE IS property_fact_table
'''

# MODELLING THE location_dim table

location_dim = real_estate_df[['addressLine1','addressLine2', 'city', 'state', 'zipCode', 'formattedAddress', 'county', 'longitude', 'latitude']].copy().drop_duplicates().reset_index(drop = True)
location_dim['location_ID'] = range(1, len(location_dim) + 1)
location_dim = location_dim[['location_ID', 'addressLine1','addressLine2', 'city', 'state', 'zipCode', 'formattedAddress', 'county', 'longitude', 'latitude']].reset_index(drop = True)

location_dim.head()

,location_ID,addressLine1,addressLine2,city,state,zipCode,formattedAddress,county,longitude,latitude
0,1,511 John St,Unknown,Bryan,OH,43506,"511 John St, Bryan, OH 43506",Williams,-84.570163,41.469595
1,2,1067 Marlin Dr,Unknown,Rockledge,FL,32955,"1067 Marlin Dr, Rockledge, FL 32955",Brevard,-80.746982,28.342440
2,3,1136 Sullivan Rd,Apt 15,Ogden,UT,84403,"1136 Sullivan Rd, Apt 15, Ogden, UT 84403",Weber,-111.950330,41.204860
3,4,101 Trinity Lakes Dr,Apt 361,Sun City Center,FL,33573,"101 Trinity Lakes Dr, Apt 361, Sun City Center...",Hillsborough,-82.355936,27.710284
4,5,7843 S Euclid Ave,Apt 1,Chicago,IL,60649,"7843 S Euclid Ave, Apt 1, Chicago, IL 60649",Cook,-87.576879,41.752132


In [36]:

# MODELLING THE sales_dim table

sales_dim = real_estate_df[['lastSalePrice','lastSaleDate']].copy().drop_duplicates().reset_index(drop = True)
sales_dim['sales_ID'] = range(1, len(sales_dim) + 1)
sales_dim = sales_dim[['sales_ID', 'lastSalePrice','lastSaleDate']]

sales_dim.head()

,sales_ID,lastSalePrice,lastSaleDate
0,1,105000.0,2000-10-13T00:00:00.000Z
1,2,125000.0,2019-08-13T00:00:00.000Z
2,3,0.0,Unknown
3,4,225000.0,2022-09-23T00:00:00.000Z
4,5,0.0,2016-06-27T00:00:00.000Z


In [67]:
real_estate_df['features'] = real_estate_df['features'].astype(str) # To prevent error when dropping duplicates
real_estate_df['taxAssessment'] = real_estate_df['taxAssessment'].astype(str)
real_estate_df['propertyTaxes'] = real_estate_df['propertyTaxes'].astype(str)

In [60]:
# MODELLING THE freatures_dim table

features_dim = real_estate_df[['bedrooms', 'squareFootage', 'features', 'bathrooms', 'lotSize']].copy().drop_duplicates().reset_index(drop = True)
features_dim['features_ID'] = range(1, len(features_dim) + 1)
features_dim = features_dim[['features_ID', 'bedrooms', 'squareFootage', 'features', 'bathrooms', 'lotSize']].reset_index(drop = True)

features_dim.head()

,features_ID,bedrooms,squareFootage,features,bathrooms,lotSize
0,1,3.0,1404.0,"{'architectureType': 'Conventional', 'cooling'...",2.0,11108.0
1,2,3.0,2258.0,"{'cooling': True, 'coolingType': 'Central', 'e...",2.0,10890.0
2,3,0.0,0.0,{},0.0,0.0
3,4,6.0,3345.0,"{'architectureType': 'Two Family', 'cooling': ...",3.0,3780.0
4,5,5.0,2418.0,"{'cooling': True, 'coolingType': 'Central', 'f...",2.5,5053.0


In [68]:

# MODELLING THE property_fact_table 

'''For the fact table we will start with building the table with the data present in real_estate_df
then join with each of the dimension tables to get the FKs and select relevant columns from the joined table'''

#Building the table with the data present in real_estate_df

property_fact_table = real_estate_df[['id', 'yearBuilt','assessorID', 'features','formattedAddress', 'lastSalePrice','legalDescription', 'ownerOccupied', 'propertyType', 'taxAssessment', 'propertyTaxes', 'subdivision', 'zoning']].copy().reset_index(drop = True)

#Join with each of the dimension tables to get the FKs

property_fact_table = property_fact_table.merge(location_dim[['location_ID', 'formattedAddress']], on = 'formattedAddress', how = 'left')
property_fact_table = property_fact_table.merge(sales_dim[['sales_ID','lastSalePrice']], on = 'lastSalePrice', how = 'left')
property_fact_table = property_fact_table .merge(features_dim[['features_ID', 'features']], on = 'features', how = 'left')

# SELECT columns needed from the table

property_fact_table = property_fact_table[['id', 'location_ID', 'sales_ID', 'features_ID','yearBuilt','assessorID','legalDescription', 'ownerOccupied', 'propertyType', 'taxAssessment', 'propertyTaxes', 'subdivision', 'zoning']]

property_fact_table.head()

,id,location_ID,sales_ID,features_ID,yearBuilt,assessorID,legalDescription,ownerOccupied,propertyType,taxAssessment,propertyTaxes,subdivision,zoning
0,"511-John-St,-Bryan,-OH-43506",1,1,1,1959.0,063-190-07-006.000,LOT 36 CONNINS HOME ACRES 030-11740-000,1.0,Single Family,"{'2022': {'value': 36510, 'land': 10820, 'impr...",{'2021': {'total': 1799}},NaN,Unknown
1,"1067-Marlin-Dr,-Rockledge,-FL-32955",2,2,2,1966.0,25-36-05-52-00008.0-0010.00,MARLIN SUBD LOT 10 BLK 8,0.0,Single Family,"{'2021': {'value': 104510, 'land': 25000, 'imp...","{'2021': {'total': 1830}, '2022': {'total': 21...",MARLIN SUBD,R2
2,"1136-Sullivan-Rd,-Apt-15,-Ogden,-UT-84403",3,3,3,0.0,Unknown,Unknown,0.0,Unknown,Unknown,Unknown,NaN,Unknown
3,"1136-Sullivan-Rd,-Apt-15,-Ogden,-UT-84403",3,3,21,0.0,Unknown,Unknown,0.0,Unknown,Unknown,Unknown,NaN,Unknown
4,"1136-Sullivan-Rd,-Apt-15,-Ogden,-UT-84403",3,3,25,0.0,Unknown,Unknown,0.0,Unknown,Unknown,Unknown,NaN,Unknown


### DATA LOADING

In [13]:
# Develop a function to get the database connections
def get_db_connection():
    connection = psycopg2.connect(
        host = 'localhost',
        database = 'zuma_realtors',
        user = 'postgres',
        password = 'Grateful@1'

    )
    return connection

# connect to our database
conn = get_db_connection()

In [75]:
# Create a function that sets up the schema and tables

def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    create_table_query = '''
    CREATE SCHEMA IF NOT EXISTS zuma_realtors;

    DROP TABLE IF EXISTS zuma_realtors.property_fact_table CASCADE;
    DROP TABLE IF EXISTS zuma_realtors.features_dim CASCADE;
    DROP TABLE IF EXISTS zuma_realtors.sales_dim CASCADE;
    DROP TABLE IF EXISTS zuma_realtors.location_dim CASCADE;

    CREATE TABLE zuma_realtors.location_dim (
        location_id INTEGER PRIMARY KEY,
        addressLine1 VARCHAR(10000),
        city VARCHAR(1000),
        state VARCHAR(100),
        zipcode INTEGER,
        formattedAddress VARCHAR(10000),
        county VARCHAR(1000),
        longitude FLOAT,
        latitude FLOAT,
        addressLine2 VARCHAR(10000)
    );

    CREATE TABLE zuma_realtors.sales_dim (
        sales_id INTEGER PRIMARY KEY,
        lastSalePrice FLOAT,
        lastSaleDate VARCHAR(255)
    );

    CREATE TABLE zuma_realtors.features_dim (
        features_id INTEGER PRIMARY KEY,
        bedrooms FLOAT,
        squareFootage FLOAT,
        features VARCHAR(10000),
        bathrooms FLOAT,
        lotsize FLOAT
    );

    CREATE TABLE zuma_realtors.property_fact_table (
        id VARCHAR(100000),
        location_ID INTEGER,
        sales_ID INTEGER,
        features_ID INTEGER,
        yearBuilt FLOAT,
        assessorID VARCHAR(255),
        legalDescription VARCHAR(10000),
        ownerOccupied FLOAT,
        propertyType VARCHAR(1000),
        taxAssessment VARCHAR(10000),
        propertyTaxes VARCHAR(1000),
        subdivision VARCHAR(1000),
        zoning VARCHAR(1000),
        FOREIGN KEY (location_ID) REFERENCES zuma_realtors.location_dim(location_id),
        FOREIGN KEY (sales_ID) REFERENCES zuma_realtors.sales_dim(sales_id),
        FOREIGN KEY (features_ID) REFERENCES zuma_realtors.features_dim(features_id)
    );
    '''

    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()

In [76]:
create_tables()

In [78]:
## LOADING THE DATA INTO THE DATABASE TABLES

conn = get_db_connection()
cursor = conn.cursor()

# Insert the dataframe into SQL tables

for _,row, in location_dim.iterrows():
    cursor.execute(
      '''INSERT INTO zuma_realtors.location_dim (location_ID, addressLine1, city, state, zipCode, formattedAddress, county, longitude, latitude, addressLine2)
      VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''',
      (row['location_ID'], row['addressLine1'], row['city'], row['state'], row['zipCode'], row['formattedAddress'], \
        row['county'], row['longitude'], row['latitude'], row['addressLine2']) 
    )

for _,row, in sales_dim.iterrows():
    cursor.execute(
      '''INSERT INTO zuma_realtors.sales_dim (sales_ID, lastSalePrice, lastSaleDate)
      VALUES (%s, %s, %s)''',
      (row['sales_ID'], row['lastSalePrice'], row['lastSaleDate']) 
    )

for _,row, in features_dim.iterrows():
    cursor.execute(
      '''INSERT INTO zuma_realtors.features_dim (features_ID, bedrooms, squareFootage, features, bathrooms, lotSize)
      VALUES (%s, %s, %s, %s, %s, %s)''',
      (row['features_ID'], row['bedrooms'], row['squareFootage'], row['features'], row['bathrooms'], row['lotSize']) 
    )

for _,row, in property_fact_table.iterrows():
    cursor.execute(
      '''INSERT INTO zuma_realtors.property_fact_table (id, location_ID, sales_ID, features_ID, yearBuilt, assessorID, legalDescription, ownerOccupied, propertyType, taxAssessment, propertyTaxes, subdivision, zoning)
      VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''',
      (row['id'], row['location_ID'], row['sales_ID'], row['features_ID'], row['yearBuilt'], row['assessorID'], \
        row['legalDescription'], row['ownerOccupied'], row['propertyType'], row['taxAssessment'], row['propertyTaxes'], row['subdivision'], row['zoning']) 
    )   


# Commit Changes
conn.commit()

# Close Connection
cursor.close()
conn.close()

